In [6]:
import os
import polars as pl
import altair as alt

In [187]:
pl.Config.set_tbl_rows(300)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [8]:
df = pl.read_parquet(os.path.join("data","projevy.parquet")).sort(by='datum')

In [4]:
df.filter(pl.col("text").str.contains("(?i)čmýr")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool
"""Poslanec Petr Kudela""","""2013ps_stenprot_027schuz_s027122.htm""","""/sqw/detail.sqw?id=6217""","""Vážený pane nepřítomný ministře zdravotnictví, budoucnost naší společnosti spočívá v dětech a v jejich přípravě na to, aby převzaly naši úlohu. Děti m…",3,"""ps""",2013,27,122,2015-04-30,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""27. schůze""",true,true,true


In [5]:
df.filter(pl.col("text").str.contains("(?i)menstru")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool
"""Posl. Krejčí""","""1925ns_ps_stenprot_174schuz_s174006.htm""",null,"""Slavná sněmovno! Vypadá to skoro komicky, že mezi paní poslance, které projednávají zákon o porodních asistentkách, vplétá se najednou poslanec muž. A…",2,"""ns""",1925,174,6,1928-11-08,"""Národní shromáždění československé / Poslanecká sněmovna1925 – 1929 Poslanecká sněmovna""","""174. schůze""",true,true,true
"""Posl. Hatina""","""1929ns_ps_stenprot_189schuz_s189002.htm""",null,"""Slavná sněmovno! Každý, kdo objektivně posuzuje projednávanou osnovu zákona o nemocech z povolání, musí doznati, že tato osnova, kterou schválíme, zna…",5,"""ns""",1929,189,2,1932-05-25,"""NS RČS 1929-1935 Poslanecká sněmovna""","""189. schůze""",true,true,true
"""Poslanec Jozef Kochan""","""2002ps_stenprot_051schuz_s051107.htm""",null,"""(16.20 hodin) (pokračuje Kochan) 4. Nevím, kdo reprezentuje značnou část odborné veřejnosti, která považuje výdej bez předpisu za prospěšnou. Jediným …",1,"""ps""",2002,51,107,2005-11-24,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""51. schůze""",true,true,false
"""Místopředseda PSP Lubomír Zaorálek""","""2010ps_stenprot_022schuz_s022021.htm""","""/sqw/detail.sqw?id=252""","""Otevírám obecnou rozpravu. Nemám žádné přihlášky do obecné rozpravy, takže ji končím. Otevírám podrobnou rozpravu. Požádal bych paní poslankyni Miladu…",5,"""ps""",2010,22,21,2011-08-30,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""22. schůze""",true,true,true
"""Poslanec Petr Kudela""","""2013ps_stenprot_027schuz_s027122.htm""","""/sqw/detail.sqw?id=6217""","""Vážený pane nepřítomný ministře zdravotnictví, budoucnost naší společnosti spočívá v dětech a v jejich přípravě na to, aby převzaly naši úlohu. Děti m…",3,"""ps""",2013,27,122,2015-04-30,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""27. schůze""",true,true,true


In [6]:
df.filter(pl.col("text").str.contains("(?i)dozimet")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool
"""Společný zpravodaj výborů Sněmovny národů, místopředseda SN V. Vedra""","""1976fs_sl_stenprot_005schuz_s005003.htm""",null,"""V oblasti radiolokační a radiokomunikační techniky dochází k zastavení vývoje a výroby krátkovlnných výkonných vysílačů všech druhů a výroba se zaměřu…",1,"""fs""",1976,5,3,1980-10-21,"""FS ČSSR 1976-1980""","""5. schůze""",true,true,false
"""Poslanec SL J. Suchánek""","""1990fs_slsn_stenprot_017schuz_s017110.htm""",null,"""Vážený pane předsedající, dámy a pánové, pan generální prokurátor nás zde dojímá slovy, že na základě přijetí lustračního zákona odejde z Generální pr…",6,"""fs""",1990,17,110,1991-10-08,"""Federální shromáždění ČSFR, 1990 – 1992 Společné schůze SL a SN""","""17. schůze""",true,true,true
"""Poslanec Jaroslav Palas""","""1998ps_stenprot_026schuz_s026115.htm""","""/sqw/p.sqw?P=543""","""Pane předsedající, vážená vládo, dámy a pánové, dovolte mi, abych po dohodě s předkladatelem a zpravodajem k tomuto bodu předložil několik pozměňovací…",3,"""ps""",1998,26,115,2000-06-29,"""Parlament České republiky, Poslanecká sněmovna 1998 – 2002""","""26. schůze""",true,true,true
"""Poslanec Aleš Rozehnal""","""2002ps_stenprot_033schuz_s033080.htm""",null,"""(9.50 hodin) (pokračuje Rozehnal) Využití této suroviny na financování krvavých konfliktů je zcela mimo realitu i vzhledem k množství této suroviny do…",1,"""ps""",2002,33,80,2004-06-17,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""33. schůze""",true,true,false
"""Poslanec Vladimír Zlínský""","""2021ps_stenprot_009schuz_s009234.htm""","""/sqw/detail.sqw?id=6795""","""Vážená paní předsedající, děkuji za slovo. Vážený pane premiére, mohl byste vzhledem k možným rizikům jaderného konfliktu na našem území nebo v našem …",7,"""ps""",2021,9,234,2022-03-03,"""PČR, PS 2021-...""","""9. schůze""",true,true,true


In [7]:
df.filter(pl.col("text").str.contains("(?i)vlastizrád")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool
"""Zpravodaj posl. Krouský""","""1918ns_ps_stenprot_042schuz_s042006.htm""",null,"""Koncept tohoto trestního oznámení a ostatní práce z téhož dne psány jsou, jak zjistil znalec písma, kterého si imunitní výbor vyžádal, žalobcem p. Jan…",1,"""ns""",1918,42,6,1919-04-03,"""Národní shromáždění československé 1918 – 1920""","""42. schůze""",true,true,false
"""Zpravodaj posl. ing. Bečka""","""1918ns_ps_stenprot_059schuz_s059009.htm""",null,"""Slavné Národní Shromáždění! Ujímám se slova v tomto okamžiku, mohu říci, do duše rozrušen! (Výborně!) A jsem přesvědčen, velevážení, že nebylo potřebí…",11,"""ns""",1918,59,9,1919-06-25,"""Národní shromáždění československé 1918 – 1920""","""59. schůze""",true,true,true
"""Poslanec dr. Kolísek""","""1918ns_ps_stenprot_110schuz_s110011.htm""",null,"""Slýcháváme to, bohužel, často a s jakousi buď opovržlivostí nebo nedůvěřivosti. O uměleckém vývinu Slováků v minulosti velmi pěkně i promluvil v Turč.…",1,"""ns""",1918,110,11,1920-01-23,"""Národní shromáždění československé 1918 – 1920""","""110. schůze""",true,true,false
"""Poslanec dr. Juriga (pokračuje)""","""1920ns_ps_stenprot_005schuz_s005005.htm""",null,"""Kebyste nám neboli tak cenzurovali noviny a zavierali nás pre pravdu do žalára, nemuseli byste to tu počúvať. (Hlas: A což to, co píše Jehlička?) Čo J…",12,"""ns""",1920,5,5,1920-06-10,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925 Poslanecká sněmovna""","""5. schůze""",true,true,true
"""Sen. dr. Franta""","""1920ns_se_stenprot_005schuz_s005007.htm""",null,"""Slavný senáte! Nebudete jistě ode mne žádati, abych se podrobně zabýval německými řečmi, které byly dnes proneseny. Ale dovolte, abych se jenom u něče…",2,"""ns""",1920,5,7,1920-06-16,"""NS RČS 1920-1925 Senát""","""5. schůze""",true,true,true


In [8]:
df.filter(pl.col("text").str.contains("(?i)endiaron")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool
"""Poslankyně Věra Kovářová""","""2017ps_stenprot_040schuz_s040116.htm""","""/sqw/detail.sqw?id=6214""","""Vážený pane ministře, obracím se na vás se závažným tématem, a to je nedostupnost léků - dnes to již zaznělo - a tedy nedostupnost léků v lékárnách. P…",2,"""ps""",2017,40,116,2020-01-23,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""40. schůze""",true,true,true
"""Poslankyně Jana Mračková Vildumetzová""","""2021ps_stenprot_052schuz_s052108.htm""","""/sqw/detail.sqw?id=6471""","""Děkuji za slovo. Vážená paní místopředsedkyně, vážení členové vlády, kolegyně, kolegové, já bych si dovolila zařadit dva body. První bod s názvem Česk…",3,"""ps""",2021,52,108,2023-02-10,"""PČR, PS 2021-...""","""52. schůze""",true,true,true
"""Poslanec Andrej Babiš""","""2021ps_stenprot_071schuz_s071311.htm""","""/sqw/detail.sqw?id=6150""","""Ano. Nezvládla to. Už máte ten seznam od paní Kovářové, abychom věděli ta slova? Ale váš kolega píše na Twitter K, víte, já jsem mluvil jenom P, víte,…",3,"""ps""",2021,71,311,2023-07-26,"""PČR, PS 2021-...""","""71. schůze""",true,true,true
"""Poslanec Andrej Babiš""","""2021ps_stenprot_074schuz_s074014.htm""",null,"""Takže to je ta politika. Kdo tu rozděluje tu společnost tedy? Kdo ji rozděluje? Ale ony to mají v genech, ty tradiční strany. Já si pamatuji za ČSSD, …",1,"""ps""",2021,74,14,2023-09-05,"""PČR, PS 2021-...""","""74. schůze""",true,true,false
"""Poslanec Andrej Babiš""","""2021ps_stenprot_074schuz_s074016.htm""",null,"""Takže tohle jsme všechno dělali a samozřejmě jsme děkovali zdravotníkům, a dali jsme jim peníze, mimořádné odměny. Ve vládě jsme samozřejmě vypláceli …",1,"""ps""",2021,74,16,2023-09-05,"""PČR, PS 2021-...""","""74. schůze""",true,true,false


In [9]:
df.filter(pl.col("text").str.contains("(?i)videohr")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool
"""Senátor Petr Vícha""","""2006se_stenprot_002schuz_s002005.htm""","""/sqw/detail.sqw?id=5761&sfo=6""","""Vážený pane předsedající, milé kolegyně, vážení kolegové, je to mé první vystoupení tady v Senátu, a proto poprosím kolegy, kteří sedí nejblíže, aby k…",4,"""se""",2006,2,5,2006-12-20,"""PČR, Senát 2006-2008""","""2. schůze""",true,true,true
"""Senátor Ladislav Macák""","""2006se_stenprot_014schuz_s014005.htm""","""/sqw/detail.sqw?id=5705&sfo=6""","""Vážený pane předsedající, milé kolegyně, vážení kolegové. Chtěl bych reagovat na vystoupení kolegy Adamce. Já s ním plně souhlasím, že by bylo dobře, …",14,"""se""",2006,14,5,2008-06-04,"""PČR, Senát 2006-2008""","""14. schůze""",true,true,true
"""Ministryně obrany ČR Jana Černochová""","""2021ps_stenprot_079schuz_s079040.htm""",null,"""V květnu 2023 mě také vláda pověřila k jednání o pořízení tanků Leopard A28, jejichž nákup plánuje Spolková republika Německo. Armádě České republiky …",1,"""ps""",2021,79,40,2023-10-17,"""PČR, PS 2021-...""","""79. schůze""",true,true,false
"""Poslanec Lubomír Brož""","""2021ps_stenprot_110schuz_s110005.htm""",null,"""Já bych jenom připomenul takové důležité věci a tam dal i svoje otazníky - budeme o tom určitě mluvit na výboru a ve druhém čtení. Rozšířila se ta obl…",1,"""ps""",2021,110,5,2024-07-10,"""PČR, PS 2021-...""","""110. schůze""",true,true,false
"""Poslanec Milan Brázdil""","""2021ps_stenprot_110schuz_s110005.htm""","""/sqw/detail.sqw?id=6185""","""Děkuji za slovo. Já už jsem se pana ministra ptal, ale přece jenom do éteru se zeptám. Pobídky pro tvůrce videoher... My žijeme v době, kdy děcka jsou…",3,"""ps""",2021,110,5,2024-07-10,"""PČR, PS 2021-...""","""110. schůze""",true,true,true


In [10]:
df.filter(pl.col("text").str.contains("(?i)woke")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool


In [11]:
df.filter(pl.col("text").str.contains("(?i)narativ")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool
"""Poslanec Jan Chvojka""","""2017ps_stenprot_019schuz_s019184.htm""","""/sqw/detail.sqw?id=5926""","""Děkuji za slovo, pane předsedo. Ještě jednou hezké dopoledne, kolegyně, kolegové, vážený pane ministře. Já se přiznám, že už jsem vystupovat nechtěl, …",3,"""ps""",2017,19,184,2018-09-19,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""19. schůze""",true,true,true
"""Poslanec Dominik Feri""","""2017ps_stenprot_039schuz_s039180.htm""","""/sqw/detail.sqw?id=6449""","""Děkuji za slovo, pane místopředsedo. Vážené kolegyně, vážení kolegové, když se podíváte na vládní stanovisko u tohoto tisku, tak uvidíte, že je velmi …",3,"""ps""",2017,39,180,2019-12-03,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""39. schůze""",true,true,true
"""Poslanec Dominik Feri""","""2017ps_stenprot_044schuz_s044128.htm""","""/sqw/detail.sqw?id=6449""","""Děkuji za slovo, pane místopředsedo. Jenom velmi krátce. Snad to mohu okomentovat, poněvadž jsem z Teplic. Vláda to nezvládla nejenom po formální strá…",10,"""ps""",2017,44,128,2020-04-09,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""44. schůze""",true,true,true
"""Ministr kultury ČR Lubomír Zaorálek""","""2017ps_stenprot_049schuz_s049230.htm""",null,"""Nemám na to moc času, ale aspoň vám naznačím, v čem je ten rozdíl, a proč si nemyslím, že to je tak úplně pravda, že jde o to, že si ti občané prostě …",1,"""ps""",2017,49,230,2020-06-03,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""49. schůze""",true,true,false
"""Předseda vlády ČR Andrej Babiš""","""2017ps_stenprot_072schuz_s072105.htm""","""https://www.vlada.cz/cz/clenove-vlady/premier/zivotopis/andrej-babis-162063/""","""No děkuji za dotaz. Tak já jsem na ten dotaz novináře normálně upřímně odpověděl. Ano, nemám 51 let slezinu, to je hlavní imunitní orgán. Když mi bylo…",8,"""ps""",2017,72,105,2020-12-03,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""72. schůze""",true,true,true


## Skokani

In [48]:
interpunkce = re.compile("[" + re.escape(string.punctuation) + "]")

In [94]:
def ocisti(frejm):
    return frejm.with_columns(pl.col('text').str.replace_all(r'[^\w\s]','',).str.to_lowercase())

In [166]:
def zgrupni_a_pores_cetnost(frejm):

    frejm_pracovni = frejm.with_columns(pl.col('text').str.split(' ')).explode('text').group_by('text').len().sort(by='len',descending=True)
    celkem = frejm_pracovni.select(pl.col('len')).sum().item()
    
    return frejm_pracovni.with_columns((pl.col('len') / celkem).alias('podil'))

In [168]:
stara_snemovna = zgrupni_a_pores_cetnost(ocisti(df.filter(pl.col("obdobi") == 2017)))

In [170]:
stara_snemovna

text,len,podil
str,u32,f64
"""a""",396741,0.030054
"""to""",277547,0.021025
"""se""",253205,0.019181
"""je""",232200,0.01759
"""v""",221060,0.016746
"""že""",209714,0.015886
"""na""",187366,0.014193
"""o""",125026,0.009471
"""tak""",101140,0.007662


In [172]:
nova_snemovna = zgrupni_a_pores_cetnost(ocisti(df.filter(pl.col("obdobi") == 2021)))

In [177]:
porovnani = stara_snemovna.rename(
    {'podil':'podil_2017','len':'pocet_2017'}
).join(
    nova_snemovna.rename(
        {'podil':'podil_2021','len':'pocet_2021'}
    ), on='text', how='full'
).with_columns(
    (pl.col('podil_2021') / pl.col('podil_2017')).alias('rozdil')
)

In [179]:
porovnani

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil
str,u32,f64,str,u32,f64,f64
"""a""",396741,0.030054,"""a""",484607,0.030857,1.026705
"""to""",277547,0.021025,"""to""",373105,0.023757,1.129946
"""se""",253205,0.019181,"""se""",299110,0.019045,0.992937
"""je""",232200,0.01759,"""je""",271769,0.017305,0.983786
"""v""",221060,0.016746,"""v""",258143,0.016437,0.981552
"""že""",209714,0.015886,"""že""",244577,0.015573,0.980283
"""na""",187366,0.014193,"""na""",232213,0.014786,1.041739
"""o""",125026,0.009471,"""o""",134105,0.008539,0.901587
"""tak""",101140,0.007662,"""tak""",121187,0.007716,1.007155


In [191]:
hranice = 30
porovnani.filter((pl.col('pocet_2017') >= hranice) & (pl.col('pocet_2021') >= hranice)).sort(by='rozdil',descending=True)

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil
str,u32,f64,str,u32,f64,f64
"""zmáčkne""",36,0.000003,"""zmáčkne""",2121,0.000135,49.522349
"""2026""",35,0.000003,"""2026""",1125,0.000072,27.017648
"""ztišení""",39,0.000003,"""ztišení""",1051,0.000067,22.651719
"""2024""",148,0.000011,"""2024""",3644,0.000232,20.695681
"""ukrajině""",90,0.000007,"""ukrajině""",2215,0.000141,20.686846
"""zdražování""",33,0.000002,"""zdražování""",781,0.00005,19.892994
"""ukrajiny""",79,0.000006,"""ukrajiny""",1746,0.000111,18.577198
"""stiskne""",81,0.000006,"""stiskne""",1532,0.000098,15.897792
"""2025""",146,0.000011,"""2025""",2676,0.00017,15.406228


In [193]:
hranice = 100
porovnani.filter((pl.col('pocet_2017') >= hranice) & (pl.col('pocet_2021') >= hranice)).sort(by='rozdil',descending=True)

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil
str,u32,f64,str,u32,f64,f64
"""2024""",148,0.000011,"""2024""",3644,0.000232,20.695681
"""2025""",146,0.000011,"""2025""",2676,0.00017,15.406228
"""energií""",173,0.000013,"""energií""",3052,0.000194,14.828646
"""2023""",280,0.000021,"""2023""",4625,0.000294,13.884069
"""zvedne""",202,0.000015,"""zvedne""",2748,0.000175,11.434796
"""teďka""",103,0.000008,"""teďka""",1328,0.000085,10.83737
"""fialy""",253,0.000019,"""fialy""",2713,0.000173,9.013477
"""místopředsedkyně""",482,0.000037,"""místopředsedkyně""",5138,0.000327,8.960044
"""korespondenční""",275,0.000021,"""korespondenční""",2876,0.000183,8.790615
